In [0]:
%run "../includes/configuration"

In [0]:
%run
"../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
#movies_df = spark.sql("select * from movie_silver.movies").filter(f"file_date = '{v_file_date}'")
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
from pyspark.sql.functions import count, countDistinct, sum, desc, lit

In [0]:
#country_df = spark.sql("select * from movie_silver.countries").filter(f"file_date = '{v_file_date}'")
#production_country_df = spark.sql("select * from movie_silver.production_country").filter(f"file_date = '{v_file_date}'")
country_df =  spark.read.format("delta").load(f"{silver_folder_path}/countries") 
production_country_df = spark.read.format("delta").load(f"{silver_folder_path}/production_country") \
                    .filter(f"file_date = '{v_file_date}'")
                   


In [0]:
country_prod_coun_df = country_df.join(production_country_df, 
                                       country_df.country_id == production_country_df.country_id, 
                                       "inner") \
                                    .select(country_df.country_name, country_df.country_id, production_country_df.movie_id)

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
results_movies_country_prod_coun_df = movies_filter_df.join(country_prod_coun_df, 
                                                            movies_filter_df.movie_id == country_prod_coun_df.movie_id, 
                                                            "inner")

In [0]:
result_df = results_movies_country_prod_coun_df.select(movies_filter_df.movie_id,                                                  
                                                       results_movies_country_prod_coun_df.country_id,"year_release_date","country_name","budget","revenue")

In [0]:
results_groups_by_df = result_df.groupBy("movie_id","country_id","year_release_date","country_name") \
                    .agg(
                        sum("budget").alias("total_budget") \
                        ,sum("revenue").alias("total_revenue")
                    )

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, rank

results_dense_rank_df = Window.partitionBy("year_release_date").orderBy(
                                                                desc("total_budget"),
                                                                desc("total_revenue")
                                                                )
final_df = results_groups_by_df.withColumn("dense_rank", dense_rank().over(results_dense_rank_df)) \
                                .withColumn("created_date", lit(v_file_date))

        

In [0]:
#overwrite_partition (final_df, "movie_gold", "results_group_movie_country", "created_date") 

In [0]:
#final_df.write.mode("overwrite").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_group_movie_country")

### Delta

In [0]:
merge_key = 'tgt.country_id = src.country_id and tgt.movie_id = src.movie_id '
merge_delta_data(final_df,"/mnt/moviehistory7809/gold","movie_gold", "results_group_movie_country", "created_date", merge_key)


In [0]:
%sql
select created_date,count(*) 
from movie_gold.results_group_movie_country
group by created_date